Импортируем библиотеки

In [27]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import torch
import transformers
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DistilBertTokenizer, DistilBertForSequenceClassification

Считываем данные

In [28]:
df = pd.read_csv("https://lms.skillfactory.ru/asset-v1:Skillfactory+URFUML22p3s+2023_SEP+type@asset+block@data.csv")
df.to_parquet("data/hw1_data.parquet")

In [29]:
data = pd.read_parquet("data/hw1_data.parquet")
data

,id,target,comment_text,severe_toxicity,obscene,identity_attack,insult,threat,asian,atheist,...,article_id,rating,funny,wow,sad,likes,disagree,sexual_explicit,identity_annotator_count,toxicity_annotator_count
0,59856,0.893617,haha you guys are a bunch of losers.,0.021277,0.000000,0.021277,0.872340,0.0000,0.0,0.0,...,2006,rejected,0,0,0,1,0,0.000000,4,47
1,239607,0.912500,Yet call out all Muslims for the acts of a few...,0.050000,0.237500,0.612500,0.887500,0.1125,0.0,0.0,...,26670,approved,0,0,0,1,0,0.000000,4,80
2,239612,0.830769,This bitch is nuts. Who would read a book by a...,0.107692,0.661538,0.338462,0.830769,0.0000,0.0,0.0,...,26674,rejected,0,0,0,0,0,0.061538,4,65
3,240311,0.968750,You're an idiot.,0.031250,0.062500,0.000000,0.968750,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.000000,0,32
4,240329,0.900000,Who cares!? Stark trek and Star Wars fans are ...,0.100000,0.200000,0.000000,0.900000,0.0000,NaN,NaN,...,32846,rejected,0,0,0,0,0,0.300000,0,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90897,957507,0.166667,Methinks Bishop Braxton doth protest too much ...,0.000000,0.000000,0.166667,0.166667,0.0000,0.0,0.0,...,166439,approved,0,0,0,6,0,0.000000,10,6
90898,5363032,0.000000,Sounds pretty speculative to me. But i'm a sp...,0.000000,0.000000,0.000000,0.000000,0.0000,NaN,NaN,...,340892,approved,3,0,0,1,0,0.000000,0,4
90899,5910478,0.166667,Seriously!\nVery proud of our 'domestic progra...,0.000000,0.000000,0.166667,0.000000,0.0000,NaN,NaN,...,374717,approved,0,0,0,0,1,0.000000,0,6
90900,1052094,0.000000,Hawaii food is mostly GMO loaded with chemical...,0.000000,0.000000,0.000000,0.000000,0.0000,NaN,NaN,...,315373,approved,0,0,0,0,1,0.000000,0,6


In [30]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90902 entries, 0 to 90901
Data columns (total 45 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   id                                   90902 non-null  int64  
 1   target                               90902 non-null  float64
 2   comment_text                         90902 non-null  object 
 3   severe_toxicity                      90902 non-null  float64
 4   obscene                              90902 non-null  float64
 5   identity_attack                      90902 non-null  float64
 6   insult                               90902 non-null  float64
 7   threat                               90902 non-null  float64
 8   asian                                21687 non-null  float64
 9   atheist                              21687 non-null  float64
 10  bisexual                             21687 non-null  float64
 11  black                       

Установка random seed

In [31]:
np.random.seed(0)

In [32]:
comments = data["comment_text"]
data["target"] = (data["target"]>0.7).astype(int)
data["target"]

0        1
1        1
2        1
3        1
4        1
        ..
90897    0
90898    0
90899    0
90900    0
90901    0
Name: target, Length: 90902, dtype: int64

# Задание 1

Разделить даннные на train (0.7) и test (0.3). 

In [33]:
X, y = comments, data["target"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)

# Задание 2

Векторизация

In [34]:
# введём максимальное количество признаков вручную, так как при автоматическом определении test и train расходятся
vct = CountVectorizer(max_features=5000, stop_words=["english"])

In [35]:
X_train_vectorized = vct.fit_transform(X_train).toarray()
X_test_vectorized = vct.transform(X_test).toarray()

In [36]:
X_train_vectorized[1]

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
print(X_train_vectorized[2].shape)
print(X_test_vectorized[2].shape)

(5000,)
(5000,)


# Задание 3

Обучим модель логистической регресии

In [38]:
model = LogisticRegression(
    max_iter=2000
    )


model.fit(
    X_train_vectorized,
    y_train
)

LogisticRegression(max_iter=2000)

#

In [39]:
y_pred = model.predict(X_test_vectorized)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy модели: {accuracy:.4f}")

Accuracy модели: 0.9221


Модель работает хорошо

# Задание 4

Функция

In [40]:
def predict_toxicity(comment: str) -> float:
    comment_array = vct.transform(np.array([comment]))
    return model.predict_proba(comment_array)[0][1]

# Задание 5

Проверка на яблоках

In [41]:
print(f'I love apples: {predict_toxicity("I love apples"):.4f}')
print(f'Apples are stupid: {predict_toxicity("Apples are stupid"):.4f}')

I love apples: 0.1228
Apples are stupid: 0.9996


# Задание 6
Топ 10к по токсичности

In [42]:
feature_names = vct.get_feature_names_out()
coefficients = model.coef_.ravel()

import pandas as pd
df_coefficients = pd.DataFrame({'word': feature_names, 'coefficient': coefficients})
top_toxic_words = df_coefficients.sort_values(by='coefficient', ascending=False).head(10)

print("Топ-10 самых токсичных слов:")
print(top_toxic_words)

Топ-10 самых токсичных слов:
           word  coefficient
4326     stupid     9.292791
2232     idiots     8.873782
2230      idiot     8.529892
4328  stupidity     7.696344
2231    idiotic     7.240594
3238   pathetic     6.510420
2215  hypocrite     6.422445
2918      moron     6.295186
1820      fools     6.276093
1430       dumb     6.201317


# Задание 7

Все слова в топ 10 действительно токсичные и оскорбительные

# Задание 8

Предвзятость

In [43]:
# Тестирование комментариев
comments_to_test = [
    "I have a christian friend",
    "I have a muslim friend",
    "I have a white friend",
    "I have a black friend"
]

for comment in comments_to_test:
    toxicity_prob = predict_toxicity(comment)
    print(f"Комментарий: {comment}")
    print(f"Вероятность токсичности: {toxicity_prob:.4f}\n")

Комментарий: I have a christian friend
Вероятность токсичности: 0.1763

Комментарий: I have a muslim friend
Вероятность токсичности: 0.4507

Комментарий: I have a white friend
Вероятность токсичности: 0.3370

Комментарий: I have a black friend
Вероятность токсичности: 0.5352



Комментарии с упоминанием мусульман и чернокожих людей имеют более высокую вероятность быть помеченными как токсичные, чем комментарии о христианах или белых людях. Это может быть связано с тем, что в обучающих данных слова "muslim" и "black" чаще встречались в контексте негатива или оскорблений.

Модель плохо работает с контекстом. Например, нейтральный комментарий вроде "I have a muslim friend" может быть ошибочно интерпретирован как токсичный из-за bias'a в данных. Это нарушает принцип справедливости, так как комментарии, связанные с определенными группами, классифицируются строже, чем другие.

# Задания 10 и 11

In [44]:
# Проверка поддержки MPS
if torch.backends.mps.is_available():
    print("using metal")
    device = torch.device("mps")
else:
    print("using cpu")
    device = torch.device("cpu")

using metal


In [45]:
X_train, X_test, y_train, y_test = train_test_split(
    comments, data["target"], test_size=0.3, random_state=0, stratify=data["target"]
)

In [46]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
model.to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [47]:
def tokenize(texts):
    return tokenizer(
        texts.tolist(),
        padding=True, 
        truncation=True, 
        max_length=64, 
        return_tensors="pt"
    ).to(device)

In [48]:
train_encodings = tokenize(X_train)
test_encodings = tokenize(X_test)

In [49]:
class ToxicityDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

train_dataset = ToxicityDataset(train_encodings, y_train.tolist())
test_dataset = ToxicityDataset(test_encodings, y_test.tolist())

In [50]:
def compute_metrics(pred):
    logits, labels = pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels, predictions)}

In [51]:
# Параметры обучения
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    report_to='none'                  # Или удалите эту строку, если не используете логгеры
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

In [52]:
trainer.train()

/opt/anaconda3/envs/arima-env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy
1,0.212000,0.204552,0.925782
2,0.128600,0.210075,0.924205
3,0.064800,0.307337,0.925012


/opt/anaconda3/envs/arima-env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/opt/anaconda3/envs/arima-env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=5967, training_loss=0.14660312703092512, metrics={'train_runtime': 1512.0069, 'train_samples_per_second': 126.251, 'train_steps_per_second': 3.946, 'total_flos': 3160887391454976.0, 'train_loss': 0.14660312703092512, 'epoch': 3.0})

In [53]:
results = trainer.evaluate()
print(f"Accuracy модели BERT: {results['eval_accuracy']:.4f}")

/opt/anaconda3/envs/arima-env/lib/python3.11/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Accuracy модели BERT: 0.9258


In [58]:
def predict_toxicity_bert(comment):
    # Токенизация
    inputs = tokenizer(
        comment,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=64
    )
    
    # Перемещение данных на MPS (если доступно)
    if torch.backends.mps.is_available():
        inputs = {key: val.to("mps") for key, val in inputs.items()}
    
    # Предсказание
    outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=1)
    return probs.detach().cpu().numpy()[0][1]  # Перемещаем результат обратно на CPU

In [59]:
comments_to_test = [
    "I have a christian friend",
    "I have a muslim friend",
    "I have a white friend",
    "I have a black friend"
]

for comment in comments_to_test:
    prob = predict_toxicity_bert(comment)
    print(f"Комментарий: {comment}")
    print(f"Вероятность токсичности: {prob:.4f}\n")

Комментарий: I have a christian friend
Вероятность токсичности: 0.0032

Комментарий: I have a muslim friend
Вероятность токсичности: 0.9823

Комментарий: I have a white friend
Вероятность токсичности: 0.9434

Комментарий: I have a black friend
Вероятность токсичности: 0.9993



Эксперимент со сменой модели в целом прошёл не успешно

1
